<a href="https://colab.research.google.com/github/Remydeme/Descarte/blob/master/TFIDF_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install seaborn 
!pip install spacy 
!pip install pyldavis
!pip install bokeh
# download the vocab  
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip install tqdm
!pip install eli5
!pip install lime
!pip install skater
!pip install --upgrade gensim
!pip install shap
!pip install plotly 
!pip install chart_studio
!pip install lime

# Introduction 


Dans ce Notebook nous allons étudier le TF-IDF de manière approfondie. Nous travaillerons sur le dataset 20 news groups.

Goals : 

* Dans ce notebook nous avons étudier comment configurer le TFidfVectorizer. 

* Etudier dans la partie entrainement, 
  * l'impact des ngram_range
  * l'impact sur la précision des prédictions, de la fusion des matrices des deux types de vectorizer.

* Étudier l'impact du paramètre max_feature. 

* Valider nos arguments sur le dataset FakeNews

Résultats : Nous avons valider notre modèle sur le dataset Kaggle FakeNews est obtenu 99.2 % de précision avec notre meilleur modèle. Ce qui nous classe premier de cette petite compétition.

Note : Ce notebook est très long 😬. Vous pouvez directement à l'aide du sommaire aller lire notre conclusion sur le TF-IDF.

# TF-IDF 

## Fonctionnement 

Le tf-idf régle un des problèmes majeur de l'algorithme du BAG-OF-WORD. Admettons que nous souhaitions classifier la phrase "Jule aime les pommes", dans la catégorie **"sentiment positif"**. 

Notre corpus de texte est : 

"Jule **aime** les pommes." (texte à classifier)
"Jule **aime** le jaguar c'est un belle animal."
"Camille va souvant à la plage elle **aime** bronzer."

1. l'algorithme va compter le nombre de mot dans le texte.
2. On va déterminer l'importance d'un mot par sa fréquence d'apparition dans le texte. 
3. On entraine notre modèle sur ces matrice représentant nos textes.
4. Notre modèle classife le texte. 

Ici, le problème qui se présente, c'est que tous les mots ont la même fréquence d'apparition dans le texte, 1 fois. Il n'est donc pas possible de dire que le mot **" aime "** est plus important que les autres mots présents dans le texte. 

### Term Frequency and Inverse Document Frequency 

* TF : compte le nombre d'apparition de chaque mot dans le corpus. Il mesure donc l'importance du mot dans le corpus. La probabilité du mot dans le texte.

* IDF : Mesure l'importance de chaque mot dans tout le corpus de textes. Il répond à la question : Est ce que ce mot est un théme dans les documents ?  


* TF : Nombre de fois mot apparait dans le texte / nombre de mots dans le texte
![Texte alternatif…](https://miro.medium.com/proxy/1*HM0Vcdrx2RApOyjp_ZeW_Q.png)

* IDF : log(Nombre de document dans lequel le mot apparait / nombre de document)  

![Texte alternatif…](https://miro.medium.com/proxy/1*A5YGwFpcTd0YTCdgoiHFUw.png)

* TF-IDF = TF * IDF 

![Texte alternatif…](https://miro.medium.com/proxy/1*nSqHXwOIJ2fa_EFLTh5KYw.png)



## Objectif 

1. Déterminer si l'utilisation de deux TF-IDF, un pour les charactère et un autre pour les mots, améliore la précision de notre modèle. 

2. Déterminer comment bien configurer notre TF-IDF. Nous  allons nous concentrer sur le ngram_range et max_feature. 

3. Étudier l'impacte de la taille du vocabulaire. 





In [0]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    return out 

In [0]:
news = twenty_newsgroup_to_csv()

In [0]:
news = news[['text', 'target', 'title']]

In [0]:
distribution = news.title.value_counts()
distribution = pd.DataFrame({'Category' : distribution.index, 'Frequency' : distribution.values})
distribution

,Category,Frequency
0,rec.sport.hockey,600
1,soc.religion.christian,599
2,rec.motorcycles,598
3,rec.sport.baseball,597
4,sci.crypt,595
5,sci.med,594
6,rec.autos,594
7,comp.windows.x,593
8,sci.space,593
9,comp.os.ms-windows.misc,591


,Category,Frequency
0,rec.sport.hockey,600
1,soc.religion.christian,599
2,rec.motorcycles,598
3,rec.sport.baseball,597
4,sci.crypt,595
5,sci.med,594
6,rec.autos,594
7,comp.windows.x,593
8,sci.space,593
9,comp.os.ms-windows.misc,591


In [0]:
import plotly.express as px 

px.bar(distribution, x="Frequency", y='Category', color='Category', orientation='h', labels={'Category': 'Thème', 'Frequency' : 'Nombre de textes'})

In [0]:
distribution.describe()

,Frequency
count,20.000000
mean,565.700000
std,58.251813
min,377.000000
25%,574.500000
50%,591.000000
75%,594.250000
max,600.000000


,Frequency
count,20.000000
mean,565.700000
std,58.251813
min,377.000000
25%,574.500000
50%,591.000000
75%,594.250000
max,600.000000


Notre jeu de données contient 20 classes différentes. Il est bien distibué. On à une une moyenne de 565 textes et un ecart-type de 58 textes. Les classe les moins bien représenté sont respectivement dans l'ordre : religion, politics et atheism. 

## Cleaning 


Avant de pouvoir entrainer nos modèle sur les textes, nous devont nettoyer (cleaning) ces textes et les mettre sous un format (token & vector) compréhensible par notre modèle. 

Voici quelques petites fonction de nettoyage standard. Nous nous aidons du package Spacy.

In [0]:
import spacy
import en_core_web_sm

print(f'Spacy version {spacy.__version__}')
nlp = en_core_web_sm.load()
stop_words = spacy.lang.en.STOP_WORDS
punctuations = spacy.lang.punctuation.LIST_PUNCT

Spacy version 2.2.4


Prépare texte :

1. met tous les caractères en minuscules 
2. applique le lemming. remplace les adjectif, verbe par un terme issu de l’usage ordinaire des locuteurs de la langue.
3. Enlève les mots fréquent "STOP WORDS" du vocabulaire anglais 
4. On enlève aussi les charactères inutile.(travail fait à la main)

In [0]:
def prepareText(text, punctuation=True, lemming=True, stop_word=True):
    """
    Prepare the text by removing punctuation, stop words and doing lemming 
    :param text: 
    :return: text 
    """     
    clean_text = nlp(text)
    
    #lowering word
    #lemming 
    # if words is pronoun don't apply lemming because spacy convert the words 
    # in "_PRON-" 
    if lemming == True:
        clean_text = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in clean_text]
    
    #remove stop words
    if stop_word == True:
        clean_text = [ word for word in clean_text if (word not in stop_words) ]
    # remove punctuation 
    if punctuation == True:
        clean_text = [word for word in clean_text if word.isalpha() ]
    
    #remove single char [b-Z] we only keep 'a'
    clean_text = [ word for word in clean_text if (len(word) != 1 and word != 'a') ]

    #clean_text = [ word for word in clean_text if (word not in noisy_words)]


    return clean_text

Cette fonction de cleaning s'occupe du nettoyage du texte en supprimant les e-mail.

In [0]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    return df

## Vocabulaire

Faisons une étude détaillés de la composition de notre jeu de données de textes. Nous allons faire une étude sur 3 catégories religions, auto, hockey. 

1. Tailles est composition du vocabulaire. 
2. Etude de la distribution (# mots par texte).

In [0]:
news  = [ news[news["title"] == "rec.sport.hockey"], news[news["title"] == "soc.religion.christian"], news[news["title"] == "rec.autos"] ]
news = pd.concat(news)

### Tailles est composition du vocabulaire

In [0]:
text_stack = " "
for text in news.text:
  text_stack += text

Nous avons fusionner tout nos texte en un. L'objectif est de déterminer la taille de notre vocabulaire. 

In [0]:
splited_text = text_stack.split(' ')

In [0]:
print(f"Le corpus de textes est composé au total de : {len(splited_text)} mots ")

Le corpus de textes est composé au total de : 410409 mots 
Le corpus de textes est composé au total de : 410409 mots 


Pour déterminer la taille de notre vocabulaire nous allons utiliser la fonction CountVectorizer. Il va analyser nos texte et construire un dictionnaire contenant le vocabulaire.

Nous lui passons en paramètre notre méthode "prepareText". Notre méthode va réduire la taille de nos textes en supprimant des élément non pertinent (mots fréquents, ponctuation ...).  

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer(tokenizer=prepareText, ngram_range=(1,1))
cvect_3 = CountVectorizer(tokenizer=prepareText, ngram_range=(1,3))

In [0]:
news = standardize_text(df=news, text_field='text')

In [0]:
corpus = news.text.to_list()
cvect.fit(corpus)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function prepareText at 0x7f82112129d8>,
                vocabulary=None)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function prepareText at 0x7f8204b08510>,
                vocabulary=None)

In [0]:
cvect_3.fit(corpus)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function prepareText at 0x7f82112129d8>,
                vocabulary=None)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function prepareText at 0x7f8204b08510>,
                vocabulary=None)

Paramètres de notre modèle: 

* analyzer='word' => pour notre tokenizer nos paramètres sont des mots.  
* dtype=<class 'numpy.int64'> => Le type de notre matrice retourner par transform() (par manque de RAM utilisé 32bit plutôt) 
* encoding='utf-8'
* input='content',    
* lowercase=True 
* max_df=1.0 
* max_features=None 
* min_df=1,
* ngram_range=(1, 1)
* preprocessor=None, stop_words=None,
* strip_accents = None
* token_pattern : => pas utilisé car on utilise notre propre tokenizer 
* tokenizer=<function prepareText at 0x7f69077e0d08>,
* vocabulary=None 

Observons le vocabulaire.

In [0]:
print(f"Notre vocabulaire contient :  {len(cvect.vocabulary_)} mots")

Notre vocabulaire contient :  15487 mots
Notre vocabulaire contient :  15487 mots


In [0]:
print(f"Le vocabulaire avec ngram_range=(1,3) contient :  {len(cvect_3.vocabulary_)} mots")

Le vocabulaire avec ngram_range=(1,3) contient :  259269 mots
Le vocabulaire avec ngram_range=(1,3) contient :  259269 mots


In [0]:
(259269) / 15487

16.74107315813263

16.74107315813263

La taille du vocabulaire augmente considérablement avec l'emploie de ngram. Ici,  avec NGRAM_range = (1,3), la taille de notre vocabulaire est  multiplié par **16.7** fois. 

In [0]:
cvect.get_feature_names()[10:30]

['abbie',
 'abbreviation',
 'abc',
 'abe',
 'abhorent',
 'abhorrent',
 'abide',
 'abideth',
 'abiding',
 'ability',
 'abiogenesis',
 'able',
 'ablility',
 'ably',
 'aboard',
 'abode',
 'abolish',
 'abolition',
 'abomination',
 'abort']

['abbie',
 'abbreviation',
 'abc',
 'abe',
 'abhorent',
 'abhorrent',
 'abide',
 'abideth',
 'abiding',
 'ability',
 'abiogenesis',
 'able',
 'ablility',
 'ably',
 'aboard',
 'abode',
 'abolish',
 'abolition',
 'abomination',
 'abort']

# Distribution du nombre de mot 

On souhaite compter le nombre de mots par texte pour chaque catégorie. Afin de déterminer si la longueur des textes a un impact sur la classification faite par notre modèle. 

 

In [0]:
def lenText(text):
  """
    Count the number of token present in a text. 
    @params text : text that we want to analyse 
  """
  return len(text.split(' '))

In [0]:
text_size_df = news.text.apply(lenText)

In [0]:
news['text_size'] = text_size_df

In [0]:
news.head()

,text,target,title,text_size
21,\nI think that Mike Foligno was the captain of...,10,rec.sport.hockey,90
35,\nFunny you should mention this one time on H...,10,rec.sport.hockey,71
57,\nNo no no!!! It's a squid! Keep the traditi...,10,rec.sport.hockey,23
88,\n \...,10,rec.sport.hockey,75
113,\n\nWell I don't see any smileys here I am t...,10,rec.sport.hockey,44


,text,target,title,text_size
21,\nI think that Mike Foligno was the captain of...,10,rec.sport.hockey,90
35,\nFunny you should mention this one time on H...,10,rec.sport.hockey,71
57,\nNo no no!!! It's a squid! Keep the traditi...,10,rec.sport.hockey,23
88,\n \...,10,rec.sport.hockey,75
113,\n\nWell I don't see any smileys here I am t...,10,rec.sport.hockey,44


In [0]:
import plotly.express as px

px.box(news, x='text_size', y='title', color='title',  orientation='h', category_orders={'title' : ['rec.autos', 'soc.religion.christian', 'rec.sport.hockey']})

Ici on peut observer que les texte pour les différentes catégories ont des longueur très différentes. 

* La catégorie automobile à des texte cours comparé aux autres catégories. 75% des textes ont une longueur inférieur à 160 tokens (avant cleaning).


Donc nos textes, avant que l'on ai appliqué le cleaning ont en moyenne 269 token. Voyons après cleaning.  

In [0]:
cleaned_text = news.text.apply(prepareText)

In [0]:
news['cleaned_text'] = cleaned_text

In [0]:
cleaned_text_size = [len(tokens) for tokens in news.cleaned_text]

In [0]:
news['cleaned_text_size'] = cleaned_text_size

In [0]:
news.cleaned_text_size.describe()

count    1793.000000
mean       79.447295
std       190.363558
min         0.000000
25%        18.000000
50%        41.000000
75%        82.000000
max      6158.000000
Name: cleaned_text_size, dtype: float64

count    1793.000000
mean       79.447295
std       190.363558
min         0.000000
25%        18.000000
50%        41.000000
75%        82.000000
max      6158.000000
Name: cleaned_text_size, dtype: float64

In [0]:
import plotly.express as px

px.box(news, x='cleaned_text_size', y='title', color='title',  orientation='h', category_orders={'title' : ['rec.autos', 'soc.religion.christian', 'rec.sport.hockey']}, labels={'title' : 'Thème', 'cleaned_text_size' : 'Nombre de mot par texte'})

In [0]:
for category in ['rec.autos', 'soc.religion.christian', 'rec.sport.hockey']:
  print(str(20 * '-') + category + str(20 * '-'))
  print(news[news['title'] == category].describe())

--------------------rec.autos--------------------
         text_size  cleaned_text_size
count   594.000000         594.000000
mean    146.872054          49.422559
std     335.808029         110.693148
min       1.000000           0.000000
25%      41.000000          15.000000
50%      78.000000          28.000000
75%     159.750000          55.750000
max    6004.000000        2064.000000
--------------------soc.religion.christian--------------------
         text_size  cleaned_text_size
count   599.000000         599.000000
mean    307.662771         103.799666
std     429.153452         126.694983
min       1.000000           0.000000
25%      86.500000          31.000000
50%     176.000000          63.000000
75%     374.500000         125.500000
max    5921.000000        1117.000000
--------------------rec.sport.hockey--------------------
          text_size  cleaned_text_size
count    600.000000         600.000000
mean     353.473333          84.860000
std     1369.577197         2

* La catégorie **christian** après cleaning a des textes contenant environ 50 % plus de mots que la classe auto et 20 %  que la classe hockey. Nos classes n'ont pas des textes de même longueur. 

* après cleaning la classe :
  * auto à perdu 2/3 de son information 
  * religion à perdu 2/3 de son information 
  * Hockey à perdu 3/4 de son information 

  

**1. Est-ce que le nombre de mots dans un texte à une impacte sur sa classification ?** 

L'algorithme du TFIDF conçoit d'abord un vocabulaire à partir de l'ensemble des textes. Le TF-IDF est calculé pour chaque mot du vocabulaire, on détermine les mots important à cette étape, et donc les "thèmes". Lorsque l'on fait une prédiction, notre algorithme se base sur la présence de ces mots (qui représentent des thèmes) dans un texte afin de le classifier dans la bonne catégorie. La prédiction de notre modèle dépendra donc des mots présent (associé au thème) dans les texte.

Toutefois, il est vrai que plus un texte est long plus il aura de mots. Probablement, plusieurs mots du thème. Et donc, notre algorithme pourra mieux le classifier. Cependant, la probabilité **d'overlap** (que plusieurs thèmes se retrouve dans le texte) sera également plus élevé.

# TF-IDF entrainement 


Avec scikit-learn, il existe deux manières de réaliser une analyse par TF-IDF :

1. soit on applique le **CountVectorizer**. Qui, conçoit le vocabulaire, applique le cleaning, transforme les mots en token et construit un tableau pour chaque texte, tableau contenant la fréquence d'apparition des mot dans le texte. Puis on applique TFidfTransformer qui se chargera de multiplier ces matrice par l'IDF **Inverse document frequency**.

2. Soit, on utilise le TFIdfVectorizer qui réalise ces deux opérations en une fois. A l'aide de la méthode **fit_transform**. 

Nous allons utiliser la deuxième méthode. 


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

**Une description concise de chacun des paramètre de la fonction :**

* encoding : str, default=’utf-8’. choix de l'encodage 

* decode_error : {‘strict’, ‘ignore’, ‘replace’} (default=’strict’) Que faire dans le cas ou un charactère n'a pas le bon encoding. i

* strip_accents : {‘ascii’, ‘unicode’, None} (default=None) Méthode utilisé pour supprimer les accents. 'ascii' est une methode rapide mais ne fonctionne que sur les charactères ASCII. 'unicode' fonctionne sur tout type de charactère mais est plus lent. 


* lowercase : bool (default=True) : Convert all character to lowercase 



* tokenizer : callable or None (default=None) Permet d'utiliser un customizer personnalisé. Il est appelé après l'étape de génération des n-grams. On peut mettre comme valeur 'word'. 

* analyzer : str, {‘word’, ‘char’, ‘char_wb’} Spécifie si la paramètre devrait être fait de mot ou de ngram de charactètres.  'char_wb' crée des ngram avec des lettre appartennant à des mots. 



* stop_words : str {‘english’}, list, or None (default=None) Liste de mots fréquents. On peut passer notre liste de mots custom.

* ngram_range : tuple (min_n, max_n), default=(1, 1)
The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.

* max_df : float in range [0.0, 1.0] or int (default=1.0) Ignore les mots ayant une fréquence supérieur au niveau définie. Exemple 0.1 Si le mots à une fréquence d'apparition supérieur à 10% dans le document il sera ces occurences seront supprimé. 

* min_dffloat in range [0.0, 1.0] or int (default=1) Ignore les mots ayant une fréquence inférieur au niveau définie. Exemple 0.1 Si le mots à une fréquence d'apparition inférieur à 10% dans le document il sera ces occurences seront supprimé. 

* max_features : int or None (default=None) Si il n'est pas égale à None, un vocabulaire considérent uniquement les max_features première valeur sera construit. 


* norm‘l1’, ‘l2’ or None, optional (default=’l2’)
Each output row will have unit norm, either: * ‘l2’: Sum of squares of vector elements is 1. The cosine similarity between two vectors is their dot product when l2 norm has been applied. * ‘l1’: Sum of absolute values of vector elements is 1. See preprocessing.normalize.

* use_idfbool (default=True)

* smooth_idfbool (default=True) 



### **Words Vectorizer**

Dans cette partie nous analysons comment configurer notre TFIDF et les résultats de ces configurations.

In [0]:
import numpy as np 

tfidf_words_3 = TfidfVectorizer(tokenizer=prepareText, max_features=20000,  ngram_range=(1,3), analyzer='word', dtype=np.float32)
tfidf_words_2 = TfidfVectorizer(tokenizer=prepareText, max_features=20000, ngram_range=(1,2) , analyzer='word', dtype=np.float32)
tfidf_words_1 = TfidfVectorizer(tokenizer=prepareText, max_features=20000, ngram_range=(1,1) , analyzer='word', dtype=np.float32)

* Tokenizer : nous appliquons notre fonction de "tokenization"
* max_feature : limite notre vocabulaire au 20k mots les plus important du vocabulaire.
* ngram_range : (1,3) nous allons construire des token de 1 à 3 mots ( analyzer='words'). 

In [0]:
texte1 = """ A DNA molecule encoding a polypeptide having at least one immunogenic determinants of the CCV spike protein, said CCV spike protein having an amino acid sequence shown in SEQ ID No. 2, 4, or 6, said polypeptide being capable of eliciting a protective immune response in a dog against CCV infection or disease."""

In [0]:
texte2 = """Dr Saif said that the veterinary community has a long experience with coronaviruses causing severe disease in domestic animals and can therefore provide assistance in the understanding the epidemiology of the disease, development of models, pathogenicity studies, and mechanisms of prevention and control for SARS."""

In [0]:
tfidf_words_3.fit([texte1, texte2])
tfidf_words_2.fit([texte1, texte2])
tfidf_words_1.fit([texte1, texte2])

In [0]:
tfidf_words_3.vocabulary_

{'acid': 0,
 'acid sequence': 1,
 'acid sequence seq': 2,
 'amino': 3,
 'amino acid': 4,
 'amino acid sequence': 5,
 'animal': 6,
 'animal provide': 7,
 'animal provide assistance': 8,
 'assistance': 9,
 'assistance understanding': 10,
 'assistance understanding epidemiology': 11,
 'capable': 12,
 'capable elicit': 13,
 'capable elicit protective': 14,
 'cause': 15,
 'cause severe': 16,
 'cause severe disease': 17,
 'ccv': 18,
 'ccv infection': 19,
 'ccv infection disease': 20,
 'ccv spike': 21,
 'ccv spike protein': 22,
 'community': 23,
 'community long': 24,
 'community long experience': 25,
 'control': 26,
 'control sars': 27,
 'coronaviruse': 28,
 'coronaviruse cause': 29,
 'coronaviruse cause severe': 30,
 'determinant': 31,
 'determinant ccv': 32,
 'determinant ccv spike': 33,
 'development': 34,
 'development model': 35,
 'development model pathogenicity': 36,
 'disease': 37,
 'disease development': 38,
 'disease development model': 39,
 'disease domestic': 40,
 'disease domest

{'acid': 0,
 'acid sequence': 1,
 'acid sequence seq': 2,
 'amino': 3,
 'amino acid': 4,
 'amino acid sequence': 5,
 'animal': 6,
 'animal provide': 7,
 'animal provide assistance': 8,
 'assistance': 9,
 'assistance understanding': 10,
 'assistance understanding epidemiology': 11,
 'capable': 12,
 'capable elicit': 13,
 'capable elicit protective': 14,
 'cause': 15,
 'cause severe': 16,
 'cause severe disease': 17,
 'ccv': 18,
 'ccv infection': 19,
 'ccv infection disease': 20,
 'ccv spike': 21,
 'ccv spike protein': 22,
 'community': 23,
 'community long': 24,
 'community long experience': 25,
 'control': 26,
 'control sars': 27,
 'coronaviruse': 28,
 'coronaviruse cause': 29,
 'coronaviruse cause severe': 30,
 'determinant': 31,
 'determinant ccv': 32,
 'determinant ccv spike': 33,
 'development': 34,
 'development model': 35,
 'development model pathogenicity': 36,
 'disease': 37,
 'disease development': 38,
 'disease development model': 39,
 'disease domestic': 40,
 'disease domest

In [0]:
print(f'La taille de notre vocabulaire est de {len(tfidf_words_3.vocabulary_)} mots')

La taille de notre vocabulaire est de 137 mots
La taille de notre vocabulaire est de 137 mots


1. Ici notre paramètre max_feature n'est pas du tout utile car notre vocabulaire n'est pas grand (< 20k mots).

2. Notre modèle à bien construit notre vocabulaire avec nos trigrams de mots.

```

 'acid': 0,
 'acid sequence': 1,
 'acid sequence seq': 2,

```


A quoi ressemble notre texte transformé. 


C'est valeur représente l'importance de chaque mot dans le texte 1. C'est la représentation de notre texte avec le TF-IDF. 

#### TFIDF appliqué sur le Texte 1

In [0]:
transformed_text_3 =  tfidf_words_3.transform([texte1])
transformed_text_2 =  tfidf_words_2.transform([texte1])
transformed_text_1 =  tfidf_words_1.transform([texte1])

In [0]:
words_3 = transformed_text_3.toarray()
words_2 = transformed_text_2.toarray()
words_1 = transformed_text_1.toarray()

In [0]:
max_3 = np.argmax(words_3[0], axis=0)
max_2 = np.argmax(words_2[0], axis=0)
max_1 = np.argmax(words_1[0], axis=0)

In [0]:
(max_3, max_2, max_1)

(18, 12, 6)

(18, 12, 6)

On obtient les index du mot importants pour nos trois vectorizers.

In [0]:
(tfidf_words_1.get_feature_names()[max_1], tfidf_words_2.get_feature_names()[max_2], tfidf_words_3.get_feature_names()[max_3])

('ccv', 'ccv', 'ccv')

('ccv', 'ccv', 'ccv')

Pour les vectorizers **ngram_range = (1,1), (1,2) et (1,3)** le mots le plus important du vocabulaire est **CCV**, mot signifiant : "un virus du genre Coronavirus".  

Dans le premier texte le mot **ccv** est mentionné 3 fois, il est en effet le sujet un thème du texte. Avec deux textes, le TFID à bien capturé le fait que le texte numéro un parle bien du coronavirus. 

#### TFIDF appliqué sur le Texte 2

In [0]:
transformed_text_3 =  tfidf_words_3.transform([texte2])
transformed_text_2 =  tfidf_words_2.transform([texte2])
transformed_text_1 =  tfidf_words_1.transform([texte2])
words_3 = transformed_text_3.toarray()
words_2 = transformed_text_2.toarray()
words_1 = transformed_text_1.toarray()
max_3 = np.argmax(words_3[0], axis=0)
max_2 = np.argmax(words_2[0], axis=0)
max_1 = np.argmax(words_1[0], axis=0)
(max_3, max_2, max_1)

(37, 25, 12)

(37, 25, 12)

In [0]:
(tfidf_words_1.get_feature_names()[max_1], tfidf_words_2.get_feature_names()[max_2], tfidf_words_3.get_feature_names()[max_3])

('disease', 'disease', 'disease')

('disease', 'disease', 'disease')

Le TFIDF nous donne le mot **disease** comme thème pour le texte numéro 2. Le texte 2, parle en effet du coronavirus provoquant des maladies sérieuse chez les animaux atteint.


**Note : Le ngram_range, permet d'ajoute de l'information par l'ajout de n-gram. On augmente la taille de notre vocabulaire. Avec un range supérieur à (1,1), c'est à dire (1, n) avec *n > 1*. Notre TF-IDF cherchera à déterminer l'importance de ces *n-grams* dans les textes.**

### Char Vectorizer 

* Ici on ajoute juste le paramètre analyzer 'char'. Notre tokenizer va concevoir notre vocabulaire en faisant des ngram(2,6) et (3,6) de caractère (tout type de charactère).

In [0]:
vect_char_3 = TfidfVectorizer(max_features=40000,  lowercase=True, analyzer='char', stop_words= 'english',ngram_range=(3,6),dtype=np.float32)
vect_char_2 = TfidfVectorizer(max_features=40000, lowercase=True, analyzer='char', stop_words= 'english',ngram_range=(2,6),dtype=np.float32)

In [0]:
vect_char_3.fit([texte1, texte2])
vect_char_2.fit([texte1, texte2])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning:

The parameter 'stop_words' will not be used since 'analyzer' != 'word'



TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=40000,
                min_df=1, ngram_range=(2, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning:

The parameter 'stop_words' will not be used since 'analyzer' != 'word'



TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=40000,
                min_df=1, ngram_range=(2, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
transformed_text_char_3 = vect_char_3.transform([texte1])
transformed_text_char_2 = vect_char_2.transform([texte1])

In [0]:
vect_char_3.vocabulary_

{' a ': 12,
 'a d': 315,
 ' dn': 97,
 'dna': 623,
 'na ': 1286,
 'a m': 326,
 ' mo': 168,
 'mol': 1229,
 'ole': 1462,
 'lec': 1158,
 'ecu': 728,
 'cul': 539,
 'ule': 1916,
 'le ': 1147,
 'e e': 655,
 ' en': 112,
 'enc': 762,
 'nco': 1310,
 'cod': 516,
 'odi': 1429,
 'din': 614,
 'ing': 1075,
 'ng ': 1342,
 'g a': 888,
 'a p': 330,
 ' po': 203,
 'pol': 1575,
 'oly': 1470,
 'lyp': 1186,
 'ype': 2012,
 'pep': 1555,
 'ept': 787,
 'pti': 1595,
 'tid': 1871,
 'ide': 1008,
 'de ': 580,
 'e h': 662,
 ' ha': 128,
 'hav': 947,
 'avi': 450,
 'vin': 1976,
 ' at': 53,
 'at ': 435,
 't l': 1801,
 ' le': 156,
 'lea': 1154,
 'eas': 708,
 'ast': 431,
 'st ': 1776,
 't o': 1805,
 ' on': 189,
 'one': 1493,
 'ne ': 1331,
 'e i': 666,
 ' im': 139,
 'imm': 1039,
 'mmu': 1217,
 'mun': 1237,
 'uno': 1932,
 'nog': 1389,
 'oge': 1450,
 'gen': 922,
 'eni': 770,
 'nic': 1362,
 'ic ': 979,
 'c d': 469,
 ' de': 86,
 'det': 602,
 'ete': 837,
 'ter': 1833,
 'erm': 813,
 'rmi': 1648,
 'min': 1206,
 'ina': 1064,
 'nan'

{' a ': 12,
 'a d': 315,
 ' dn': 97,
 'dna': 623,
 'na ': 1286,
 'a m': 326,
 ' mo': 168,
 'mol': 1229,
 'ole': 1462,
 'lec': 1158,
 'ecu': 728,
 'cul': 539,
 'ule': 1916,
 'le ': 1147,
 'e e': 655,
 ' en': 112,
 'enc': 762,
 'nco': 1310,
 'cod': 516,
 'odi': 1429,
 'din': 614,
 'ing': 1075,
 'ng ': 1342,
 'g a': 888,
 'a p': 330,
 ' po': 203,
 'pol': 1575,
 'oly': 1470,
 'lyp': 1186,
 'ype': 2012,
 'pep': 1555,
 'ept': 787,
 'pti': 1595,
 'tid': 1871,
 'ide': 1008,
 'de ': 580,
 'e h': 662,
 ' ha': 128,
 'hav': 947,
 'avi': 450,
 'vin': 1976,
 ' at': 53,
 'at ': 435,
 't l': 1801,
 ' le': 156,
 'lea': 1154,
 'eas': 708,
 'ast': 431,
 'st ': 1776,
 't o': 1805,
 ' on': 189,
 'one': 1493,
 'ne ': 1331,
 'e i': 666,
 ' im': 139,
 'imm': 1039,
 'mmu': 1217,
 'mun': 1237,
 'uno': 1932,
 'nog': 1389,
 'oge': 1450,
 'gen': 922,
 'eni': 770,
 'nic': 1362,
 'ic ': 979,
 'c d': 469,
 ' de': 86,
 'det': 602,
 'ete': 837,
 'ter': 1833,
 'erm': 813,
 'rmi': 1648,
 'min': 1206,
 'ina': 1064,
 'nan'

In [0]:
vect_char_2.vocabulary_

{' a': 15,
 'a ': 342,
 ' d': 92,
 'dn': 679,
 'na': 1401,
 ' m': 176,
 'mo': 1335,
 'ol': 1588,
 'le': 1248,
 'ec': 781,
 'cu': 590,
 'ul': 2093,
 'e ': 698,
 ' e': 115,
 'en': 830,
 'nc': 1418,
 'co': 565,
 'od': 1552,
 'di': 665,
 'in': 1137,
 'ng': 1462,
 'g ': 967,
 ' p': 214,
 'po': 1717,
 'ly': 1291,
 'yp': 2199,
 'pe': 1694,
 'ep': 852,
 'pt': 1739,
 'ti': 2037,
 'id': 1080,
 'de': 634,
 ' h': 137,
 'ha': 1019,
 'av': 490,
 'vi': 2154,
 'at': 473,
 't ': 1963,
 ' l': 167,
 'ea': 770,
 'as': 452,
 'st': 1941,
 ' o': 193,
 'on': 1614,
 'ne': 1449,
 ' i': 145,
 'im': 1127,
 'mm': 1326,
 'mu': 1349,
 'un': 2098,
 'no': 1503,
 'og': 1575,
 'ge': 1003,
 'ni': 1483,
 'ic': 1066,
 'c ': 508,
 'et': 910,
 'te': 1991,
 'er': 870,
 'rm': 1798,
 'mi': 1314,
 'an': 407,
 'nt': 1525,
 'ts': 2065,
 's ': 1838,
 'of': 1561,
 'f ': 933,
 ' t': 268,
 'th': 2009,
 'he': 1036,
 ' c': 66,
 'cc': 530,
 'cv': 595,
 'v ': 2124,
 ' s': 237,
 'sp': 1927,
 'pi': 1703,
 'ik': 1122,
 'ke': 1238,
 'pr': 172

{' a': 15,
 'a ': 342,
 ' d': 92,
 'dn': 679,
 'na': 1401,
 ' m': 176,
 'mo': 1335,
 'ol': 1588,
 'le': 1248,
 'ec': 781,
 'cu': 590,
 'ul': 2093,
 'e ': 698,
 ' e': 115,
 'en': 830,
 'nc': 1418,
 'co': 565,
 'od': 1552,
 'di': 665,
 'in': 1137,
 'ng': 1462,
 'g ': 967,
 ' p': 214,
 'po': 1717,
 'ly': 1291,
 'yp': 2199,
 'pe': 1694,
 'ep': 852,
 'pt': 1739,
 'ti': 2037,
 'id': 1080,
 'de': 634,
 ' h': 137,
 'ha': 1019,
 'av': 490,
 'vi': 2154,
 'at': 473,
 't ': 1963,
 ' l': 167,
 'ea': 770,
 'as': 452,
 'st': 1941,
 ' o': 193,
 'on': 1614,
 'ne': 1449,
 ' i': 145,
 'im': 1127,
 'mm': 1326,
 'mu': 1349,
 'un': 2098,
 'no': 1503,
 'og': 1575,
 'ge': 1003,
 'ni': 1483,
 'ic': 1066,
 'c ': 508,
 'et': 910,
 'te': 1991,
 'er': 870,
 'rm': 1798,
 'mi': 1314,
 'an': 407,
 'nt': 1525,
 'ts': 2065,
 's ': 1838,
 'of': 1561,
 'f ': 933,
 ' t': 268,
 'th': 2009,
 'he': 1036,
 ' c': 66,
 'cc': 530,
 'cv': 595,
 'v ': 2124,
 ' s': 237,
 'sp': 1927,
 'pi': 1703,
 'ik': 1122,
 'ke': 1238,
 'pr': 172

**Vectorizer avec ngram_range = (3,6)**

In [0]:
words_3 = transformed_text_char_3.toarray()

In [0]:
vect_char_3.get_feature_names()[np.argmax(words_3[0])]

'g a'

'g a'

In [0]:
vect_char_3.get_feature_names()[np.argmax(words_3[0])]

'g a'

'g a'

**Vectorizer avec ngram_range = (2,6)**


In [0]:
words_2 = transformed_text_char_2.toarray()

In [0]:
vect_char_2.get_feature_names()[np.argmax(words_2[0])]

'in'

'in'

Le mot le plus important selon notre deuxième vectorizer est **in**.

Le charVectorizer conçoit des token à partir de groupe de caractères.

Donc dans les deux cas on se retrouve avec un vocabulaire constitué de ngram de caractères. Les mots de ce vocabulaire ont une taille comprise (2,6) (ou (3,6) cas de notre autre vectorizer). 


**Le vectorizer avec le paramètre analyzer='char', conçoit un vocabulaire de token dans l'intervalle (n_gram_range) que nous lui avons donné (ici 2,6). En faisant cela, nous construisons, des groupes de caractères important que notre tokenizer paramétré avec analyzer='word' n'aurait pas pu détecter (du fait de la méthode de tokenization utilisé).**

**Ainsi, en fusionnant la matrices résultant de l'application de ce transformateur, avec celui de notre tranformateur paramétré pour les mots, nous ajoutons de l'informations que notre modèle pourrat alors exploiter.**

## **Entrainement**

Nous avons pu observer les résultats de ces deux type de vectorizer. Nous allons maintenant appliquer la combinaisons des deux vectorizer sur notre jeu de donnée et étudier les performances de nos modèles.


1. Comparez **tfidf** avec un **analyzer 'word'** seul vs **tfidf avec un analyzer 'word' + tfidf avec un analyzer 'char'**.

2. comparez l'impacte des N-Gram sur les performances.

3. L'impact du max_feature sur les performances.

### **Contruction du jeu de donnée d'entrainement**

In [0]:
corpus = news.text
target = news.target.to_list()

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(corpus, target, test_size=0.3, random_state=42, stratify=target)

* Stratify : Afin que nous ayons les même proportions de classe dans le tests set et le train set. 

### **L'impact des NGRAM sur les performances**


Nous allons garder les même paramètres que ceux utilisé précédemment. Nous feront juste des tests avec différents ngram_range. 

* (1,1
* (1,2)
* (1,3)

### **NGRAM_RANGE=(1,1)**

In [0]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [0]:
vect_word = TfidfVectorizer(analyzer='word', tokenizer=prepareText, max_features=20000, ngram_range=(1,1), dtype=np.float32)
vect_char = TfidfVectorizer(analyzer='char', stop_words='english',  max_features=40000, ngram_range=(2,6), lowercase=True, dtype=np.float32)

In [0]:
%%time 
vect_corpus_word  = vect_word.fit_transform(x_train)

CPU times: user 51.3 s, sys: 507 ms, total: 51.8 s
Wall time: 52 s


In [0]:
vect_corpus_char = vect_char.fit_transform(x_train)

In [0]:
vect_test_corpus_word = vect_word.transform(x_test)
vect_test_corpus_char = vect_char.transform(x_test)

In [0]:
from scipy import sparse

In [0]:
svc = LinearSVC()

In [0]:
vect_corpus_wc = sparse.hstack([vect_corpus_word , vect_corpus_char])

In [0]:
vect_test_corpus_wc = sparse.hstack([vect_test_corpus_word , vect_test_corpus_char])

**On va commencer par analyser les résulat avec la matrice de mots du TF-IDF(analyzer='word') seul**

In [0]:
%%time 
svc.fit(vect_corpus_word, y_train)

CPU times: user 35.2 ms, sys: 1.03 ms, total: 36.2 ms
Wall time: 44.8 ms


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
prediction_11 = svc.predict(vect_test_corpus_word)

### **Resultat SVC + TFIDF(ngram=(1,1), analyzer='word') **

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(y_test, prediction_11))

              precision    recall  f1-score   support

           7       0.91      0.96      0.93       178
          10       0.98      0.94      0.96       180
          15       0.96      0.95      0.95       180

    accuracy                           0.95       538
   macro avg       0.95      0.95      0.95       538
weighted avg       0.95      0.95      0.95       538



 On obtient un score de 95% pour notre f1-score. Ce qui est vraiment bien lorque l'on sait, que l'on a pas encore fait de "features engineering". validons le modèle avec un cross_validation.

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
scores = cross_val_score(svc, vect_corpus_word, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores, 'steps' : [1, 2, 3, 4, 5]}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation est : {np.mean(scores)}')

Le score moyen de la cross validation est : 0.9274900398406374


In [0]:
svc_wc = LinearSVC()

In [0]:
svc.fit(vect_corpus_wc, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
prediction_11_wc = svc.predict(vect_test_corpus_wc)

### **Resultat SVC + TFIDF(ngram=(1,1), analyzer='word') + TFIDF(ngram=(1,1), analyzer='char')**

In [0]:
print(classification_report(y_test, prediction_11_wc))

              precision    recall  f1-score   support

           7       0.91      0.96      0.94       178
          10       0.99      0.93      0.96       180
          15       0.95      0.96      0.95       180

    accuracy                           0.95       538
   macro avg       0.95      0.95      0.95       538
weighted avg       0.95      0.95      0.95       538



In [0]:
scores = cross_val_score(svc_wc, vect_corpus_wc, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation est : {np.mean(scores)}')

Le score moyen de la cross validation est : 0.9330677290836652


L'ajout des matrices de notre char vectorizer améliore les performances de notre modèle de 1.2%. Je tiens à préciser que le paramètres de notre modèle n'ont pas été optimisés. On peut donc espérer observer des performances beaucoup plus intéressante.   

Ici la taille de notre vocabulaire était de 15000 mots.


### **NGRAM_RANGE=(1,2)**

**Nous allons augmenter la notre range à (1,2)** Mais nous allons aussi supprimer dans un premier temps le paramètre max_feature. Afin de ne pas supprimer l'information.  

In [0]:
vect_word_2 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,2), dtype=np.float32)

In [0]:
%%time 
vec_corpus_word_2 = vect_word_2.fit_transform(x_train)

CPU times: user 48.3 s, sys: 292 ms, total: 48.6 s
Wall time: 48.7 s


In [0]:
vec_test_corpus_word_2 = vect_word_2.transform(x_test)

In [0]:
print(f'La taille du vocabulaire avec ngram_range=(1,2) : {len(vect_word_2.vocabulary_)}')

La taille du vocabulaire avec ngram_range=(1,2) : 92307


**On rappel que notre vocabulaire d'origin faisait 15000 mots après cleaning. ça taille à donc été multiplié par 6.**

In [0]:
vec_corpus_wc_2 = sparse.hstack([vec_corpus_word_2, vect_corpus_char])

In [0]:
vec_test_corpus_wc_2 = sparse.hstack([vec_test_corpus_word_2, vect_test_corpus_char])

In [0]:
svc_wc2 = LinearSVC()
svc_2 = LinearSVC()

In [0]:
svc_wc2.fit(vec_corpus_wc_2, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
svc_2.fit(vec_corpus_word_2, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
prediction_2 = svc_2.predict(vec_test_corpus_word_2)

In [0]:
prediction_2_wc = svc_wc2.predict(vec_test_corpus_wc_2)

#### **Résultat SVC + TFIDF(ngram_range =(1,2) , analyzer='word')**

---



In [0]:
print(classification_report(y_test, prediction_2))

              precision    recall  f1-score   support

           7       0.93      0.94      0.94       178
          10       0.98      0.94      0.96       180
          15       0.94      0.96      0.95       180

    accuracy                           0.95       538
   macro avg       0.95      0.95      0.95       538
weighted avg       0.95      0.95      0.95       538



In [0]:
scores_2 = cross_val_score(svc_2, vec_corpus_word_2, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores_2}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation avec les ngram est : {np.mean(scores_2)}')

Le score moyen de la cross validation avec les ngram est : 0.9282868525896413


L'exactitude de notre modéle à augmenté de 0.1 %.

#### **Résultat SVC + TFIDF(ngram_range =(1,2) , analyzer='word') + TFIDF(ngram_range =(2,6) , analyzer='char')**

In [0]:
print(classification_report(y_test, prediction_2_wc))

              precision    recall  f1-score   support

           7       0.89      0.96      0.92       178
          10       0.98      0.92      0.95       180
          15       0.96      0.94      0.95       180

    accuracy                           0.94       538
   macro avg       0.94      0.94      0.94       538
weighted avg       0.94      0.94      0.94       538



In [0]:
scores_wc2 = cross_val_score(svc_wc2, vec_corpus_wc_2, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores_wc2}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation avec les ngram=(1,2) est : {np.mean(scores_wc2)}')

Le score moyen de la cross validation avec les ngram=(1,2) est : 0.9306772908366534


Avec l'ajout des **char vectorizer** nous avons gagné 1% d'exactitude en plus que sans. En revanche, comparé au résultat de la fusion des tfidf('word') + tfidf('char') avec le ngram_range=(1,1), on observe une perte d'exactitude de 0.3%. Cela, peut s'expliquer par l'augmentation de la taille de notre vocabulaire qui a été multiplié par 6. 


Remarque : Il est possible qu'avec l'augmentation de notre vocabulaire, notre modèle perde en précision sur le jeu de donnée d'entraînement, mais qu'il généralise mieux sur des données de validation.

### NGRAM_RANGE=(1,3)

In [0]:
vect_word_3 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3), dtype=np.float32)

In [0]:
vect_corpus_3 = vect_word_3.fit_transform(x_train)

In [0]:
vect_test_corpus_3 = vect_word_3.transform(x_test)

In [0]:
print(f"La taille du vocabulaire est : {len(vect_word_3.vocabulary_)} mots ")

La taille du vocabulaire est : 181964 mots 


In [0]:
vect_corpus_wc_3 = sparse.hstack([vect_corpus_3, vect_corpus_char])

In [0]:
vec_test_corpus_wc_3 = sparse.hstack([vect_test_corpus_3, vect_test_corpus_char])

In [0]:
svc_wc3 = LinearSVC()
svc_3 = LinearSVC()

In [0]:
svc_wc3.fit(vect_corpus_wc_3, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
svc_3.fit(vect_corpus_3, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
prediction_3 = svc_3.predict(vect_test_corpus_3)

In [0]:
prediction_wc3 = svc_3.predict(vect_test_corpus_3)

#### **TFID(ngram_range=(1,3), analyzer='word')**

In [0]:
print(classification_report(y_test, prediction_3))

              precision    recall  f1-score   support

           7       0.93      0.94      0.94       178
          10       0.99      0.94      0.96       180
          15       0.93      0.96      0.95       180

    accuracy                           0.95       538
   macro avg       0.95      0.95      0.95       538
weighted avg       0.95      0.95      0.95       538



In [0]:
scores_3 = cross_val_score(svc_3, vect_corpus_3, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores_3}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation avec les ngram=(1,3) est : {np.mean(scores_3)}')

Le score moyen de la cross validation avec les ngram=(1,3) est : 0.9266932270916334


#### TFID(ngram_range=(1,3), analyzer='word') + TFID(ngram_range=(1,3), analyzer='analyzer')

In [0]:
print(classification_report(y_test, prediction_wc3))

              precision    recall  f1-score   support

           7       0.93      0.94      0.94       178
          10       0.99      0.94      0.96       180
          15       0.93      0.96      0.95       180

    accuracy                           0.95       538
   macro avg       0.95      0.95      0.95       538
weighted avg       0.95      0.95      0.95       538



In [0]:
scores_wc3 = cross_val_score(svc_wc3, vect_corpus_wc_3, y_train,scoring='accuracy')

In [0]:
px.scatter(pd.DataFrame({'accuracy' : scores_wc3}), y='accuracy')

In [0]:
print(f'Le score moyen de la cross validation avec les ngram=(1,3) est : {np.mean(scores_wc3)}')

Le score moyen de la cross validation avec les ngram=(1,3) est : 0.9306772908366534


### Conclusion NGRAM

* Afin d'améliorer les performances de notre algorithme avec les NGRAM il nous faut faire du **feature engineering**. 

* La suppression du **max_feature** nous à permis de d'améliorer énormément nos performances. Ils nous faut trouver la valeur idéal en fonction de la taille de notre vocabulaire.

## Impacte du Max_feature 

Nous allons étudier l'impacte du paramètre max_feature. Il semble être responsable des mauvaises performances que nous avons observer durant l'entrainement de notre modèle. 

In [0]:
vect_word_max_feature = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3), dtype=np.float32)

In [0]:
vocab_size_prop = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [0]:
vect_word_max_feature.fit(x_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'



TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function prepareText at 0x7f8204b08510>,
                use_idf=True, vocabulary=None)

In [0]:
vocab_size = len(vect_word_max_feature.vocabulary_)

In [0]:
import math

In [0]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, recall_score
from sklearn.model_selection import cross_val_score

In [0]:
svc = LinearSVC()
classification_report_results = {}
for prop in vocab_size_prop:
  max_feature = math.ceil(vocab_size * prop)
  vect = TfidfVectorizer(analyzer='word', tokenizer=prepareText, max_features=max_feature, ngram_range=(1,3), dtype=np.float32)
  vect_corpus = vect.fit_transform(x_train)
  vect_test_corpus = vect.transform(x_test)
  svc.fit(vect_corpus, y_train)
  prediction = svc.predict(vect_test_corpus)
  print(str( str(20 * '-') + "- max_feature : " + str(max_feature) + "| soit : " + str(prop * 100) + "% du vocabulaire" + str(20 * '-')))
  classification_report_results[max_feature] = classification_report(y_test, prediction)
  print(classification_report_results[max_feature])

--------------------- max_feature : 1820| soit : 1.0% du vocabulaire--------------------
              precision    recall  f1-score   support

           7       0.89      0.92      0.91       178
          10       0.95      0.92      0.94       180
          15       0.93      0.93      0.93       180

    accuracy                           0.93       538
   macro avg       0.93      0.93      0.93       538
weighted avg       0.93      0.93      0.93       538

--------------------- max_feature : 5459| soit : 3.0% du vocabulaire--------------------
              precision    recall  f1-score   support

           7       0.91      0.93      0.92       178
          10       0.97      0.93      0.95       180
          15       0.94      0.95      0.95       180

    accuracy                           0.94       538
   macro avg       0.94      0.94      0.94       538
weighted avg       0.94      0.94      0.94       538

--------------------- max_feature : 9099| soit : 5.0% du voc

#### **Validation croisé**

Validons nos résultat par  validation croisé. 

In [0]:
from datetime import datetime

from sklearn.linear_model import LogisticRegression

In [0]:
%%time 
import numpy as np

svc = LinearSVC()
report_max_feature = []
report_f1_score = []
report_time = []
report_vect_shape = []

for prop in vocab_size_prop:
  max_feature = math.ceil(vocab_size * prop)
  vect = TfidfVectorizer(analyzer='word', tokenizer=prepareText, max_features=max_feature, ngram_range=(1,3), dtype=np.float32)
  start = datetime.now()
  vect_corpus = vect.fit_transform(x_train)
  scores = cross_val_score(svc, vect_corpus, y_train,scoring='accuracy')
  end = datetime.now()
  report_time.append(end-start)
  report_vect_shape.append(str(vect_corpus.shape))
  report_f1_score.append(np.mean(scores))
  report_max_feature.append(max_feature)

CPU times: user 10min 14s, sys: 3.83 s, total: 10min 18s
Wall time: 10min 19s


In [0]:
report_svc = {'max feature' : report_max_feature, 'proportion' : vocab_size_prop, 'duration' : report_time, 'vector shape' : report_vect_shape,'f1-score' : report_f1_score}

In [0]:
df_svc = pd.DataFrame(report_svc)

In [0]:
df_svc

,max feature,proportion,duration,vector shape,f1-score
0,1820,0.01,00:00:47.529951,"(1255, 1820)",0.919522
1,5459,0.03,00:00:47.451275,"(1255, 5459)",0.933865
2,9099,0.05,00:00:47.455973,"(1255, 9099)",0.930677
3,14558,0.08,00:00:47.584522,"(1255, 14558)",0.931474
4,18197,0.10,00:00:47.478120,"(1255, 18197)",0.933068
5,36393,0.20,00:00:47.687334,"(1255, 36393)",0.931474
6,54590,0.30,00:00:47.495475,"(1255, 54590)",0.930677
7,72786,0.40,00:00:47.593437,"(1255, 72786)",0.930677
8,90982,0.50,00:00:47.575346,"(1255, 90982)",0.929880
9,109179,0.60,00:00:47.614264,"(1255, 109179)",0.927490


In [0]:
px.scatter(df_svc,x='max feature',y='f1-score', color='proportion', size='proportion', labels={'max feature' : "Taille du vocabulaire"}, title='Évolution de la précision en fonction de la taille du vocabulaire')

Avec 1% soit 1820 mots du vocabulaire, on obtient un score de 91.9%. En ajoutant 2% des "top words" de notre vocabulaire, soit environ 3000 mots on gagne 2.1% en précision. L'augmentation de la taille du vocabulaire n'améliore pas les performance de notre modèle, au contraire on observe une baisse des performances.

#### Evolution du temps 

In [0]:
px.scatter(df_svc,x='max feature', y='duration',  color='proportion', size='proportion', labels={'max feature' : "Taille du vocabulaire", 'duration' : "Temps d'entrainement"}, title="Évolution temps d'entrainement en fonction de la taille du vocabulaire")

**Cela n'est pas flagrant car notre jeux de donnée et de petit taille. Mais on peu obsever une augmentation du temps d'entrainement avec l'augmentation de la taille de notre vocabulaire.**


*Remarque : Reste à savoir si la puissance de calcul alloué sur colab est constante durant la période d'utilisation.*

## **Conclusion max_feature**

L'utilisation du paramètre max_feature permet :

Pros:

* Gain en précisions et pertinence de notre modèle (valeur idéal à déterminer).
* Réduction de la taille de notre vocabulaire, donc des données sur lequel notre modèle apprend. 
* Suppression de données non essentiel lors de l'entrainement. 
* Gain en temps d'entrainement (à étudier plus en détail)

Cons:

* Rechercher la bonne valeur. Je pense que la valeur dépend des données. Mais une valeur comprise entre 5K et 25K mots semble être intéressante.

Le paramètre max_feature, permet de limiter notre vocabulaire à nos top N words. C'est N mots, sont déterminés par leur importances. Calculé avec le TF-IDF. **Avec seulement 1 % du vocabulaire, soit les 1870 mots ayant les scores (tfidf) les plus élevés, on obtient 92.5 % de f1-score. Ce qui est un très bon score.**

Il est donc possible d'obtenir de très bon résultats dans nos prédictions, en utilisant un pourcentage de notre vocabulaire rassemblant uniquement les mots importants.

Lorsque l'on entraine un modèle en machine learning, on cherche une **approximation** de la fonction qui, à partir des données fournis en entré, nous donne la bonne prédiction en sortie.  Grâce au paramètre **max_feature**, nous **réduisons la taille du vocabulaire**. La matrice généré par notre vectorizer est par consequent réduite. Elle est égale (nombre de texte, taille du vocabulaire). Notre modèle est entrainé sur un volume d'informations dont, on a supprimé un pourcentage d'informations moins important. 


La suppression des mots ayant un score d'importance moins élevé permet d'améliorer les performances de notre modèle (LinearSVC) et de diminuer le temps d'entrainement du modèle. 


**Remarque: Lorsque nous avions testé l'impacte des ngram_range sur les performances de notre modèle. Nous avions remarqué, que nous perdions en précision à mesure que l'on augmentait l'interval. Avec le max_feature optimal (3%) on obtient avec le ngram_range=(1,3) un bien meilleur score que précédemment 93.3% vs 92.6%.**


## **Validation**

Nous allons maintenant mettre en application ce que nous avons étudier précédemment sur un autre jeu de donnée afin de valider nos arguments. 

* LinearSVC + TFIDF(ngram=(1,3), analyzer='word') + TFIDF(ngram=(2,6) analyzer='char')

* LinearSVC + TFIDF(ngram=(1,3), analyzer='word')

pour les deux TFIDF : 

1. Cherchez les meilleurs paramètres pour notre LinearSVC avec un randomizedSVC.
  a. max_feature = 80% du vocab 
  b.  max_feature = 10% du vocab



### Jeu de données 

Nous allons utiliser le jeu de données Fake News afin de valider nos arguments. Il s'agit d'un corpus de texte constitué de **"fake news"** et de vrais arcticles de presse.  

In [0]:
 !pip install kaggle

In [0]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"blackfalcon","key":"29253f40952a569b346807ec55795121"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [0]:
!kaggle kernels list — user YOUR_USER — sort-by dateRun

!kaggle competitions download -c fake-news

!unzip -q train.csv.zip -d .
!unzip -q test.csv.zip -d .

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user YOUR_USER — sort-by dateRun
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 15.7MB/s]
100% 37.0M/37.0M [00:00<00:00, 57.0MB/s]
100% 37.0M/37.0M [00:00<00:00, 106MB/s] 
  0% 0.00/9.42M [00:00<?, ?B/s]
100% 9.42M/9.42M [00:00<00:00, 86.6MB/s]


In [0]:
!ls 

kaggle.json  submit.csv  test.csv.zip  train.csv.zip
sample_data  test.csv	 train.csv


In [0]:
import pandas as pd 

In [0]:
train_df = pd.read_csv('train.csv')

In [0]:
test_df = pd.read_csv('test.csv')

#### Vocabulaire 

### Préparation des données

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


Pour les lignes ayant l'auteur et le titre à null. Nous allons remplacer la valuer **nan** par **"Unknow"**.

In [0]:
train_df.author.fillna("unknow", inplace=True)

In [0]:
train_df.title.fillna("unknow", inplace=True)

Nous allons supprimer les lignes sans articles

In [0]:
train_df.dropna(inplace=True)

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20761 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20761 non-null  int64 
 1   title   20761 non-null  object
 2   author  20761 non-null  object
 3   text    20761 non-null  object
 4   label   20761 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 973.2+ KB


In [0]:
corpus = train_df.text.to_list()

In [0]:
titles = train_df.text.to_list()

In [0]:
target = train_df.label.to_list()

### TF-IDF 

1. Nous allons travailler avec un vectorizer avec les paramètres par défaut 
2. Nous allons utiliser un vectorizer avec le ngram_range=(1,3) et max_feature=3%
3. Nous alons utiliser un vectorizer avec  le ngram_range=(1,3) et max_feature=80% 

#### TF-IDF default 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer_default = TfidfVectorizer(analyzer='word', tokenizer=prepareText)

In [0]:
%%time
vect_corpus_default = vectorizer_default.fit_transform(corpus)

In [0]:
print(f"La taille de notre vocabulaire était : {len(svc_default.vocabulary_)}")

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
import math

In [0]:
svc_default = LinearSVC()

In [0]:
accuracy = cross_val_score(svc_default, vect_corpus_default, y=target, scoring='accuracy', cv=3, n_jobs=-1)
recall = cross_val_score(svc_default, vect_corpus_default, y=target, scoring='recall', cv=3, n_jobs=-1)

In [0]:
print(f"La precision du modèle est {np.mean(accuracy)} | la pertinence {np.mean(recall)} ce qui nous donne un score f1 de {2* np.mean(recall)*np.mean(accuracy)/(np.mean(recall)+np.mean(accuracy))}")

La precision du modèle est 0.9497132933500261 | la pertinence 0.9570079043763254 ce qui nous donne un score f1 de 0.9533466452368962


Nous obtenons un très bon f1-score de 95%. Notre modèle est très performant reste à voir si il généralise bien. 

#### TF-IDF ngram_range=(1,3) && max_feature=1%




In [0]:
vectorizer_13 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3))

Nous avons besoin de déterminer la taille de notre vocabulaire avec le ngram_range=(1,3).

In [0]:
vectorizer_13.fit(corpus)

In [0]:
vocab_size = len(vectorizer_13.vocabulary_)
one_percent = math.ceil(vocab_size * 0.01)
octo_percent = math.ceil(vocab_size * 0.8)
print(f"La taille du vocabulaire est de {vocab_size} mots | 1% du vocaublaire : {one_percent} mots 80% du vocaublaire : {octo_percent} mots ")

La taille du vocabulaire est de 10050891 mots | 1% du vocaublaire : 100509 mots 80% du vocaublaire : 8040713 mots 


In [0]:
vectorizer_13 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3), max_features=one_percent)

In [0]:
%%time
vectorized_corpus_131 = vectorizer_13.fit_transform(corpus)

CPU times: user 43min 7s, sys: 26.2 s, total: 43min 33s
Wall time: 43min 35s


In [0]:
print(f"La taille du texte vetorize est : {vectorized_corpus_131.shape}")

La taille du texte vetorize est : (20761, 100509)


In [0]:
svc_31 = LinearSVC()

In [0]:
accuracy = cross_val_score(svc_31, vectorized_corpus_131, y=target, scoring='accuracy', cv=3, n_jobs=-1)
recall = cross_val_score(svc_31, vectorized_corpus_131, y=target, scoring='recall', cv=3, n_jobs=-1)

In [0]:
print(f"La precision du modèle est {np.mean(accuracy)} | la pertinence {np.mean(recall)} ce qui nous donne un score f1 de {2* np.mean(recall)*np.mean(accuracy)/(np.mean(recall)+np.mean(accuracy))}")

La precision du modèle est 0.9618513952815689 | la pertinence 0.9679005205320994 ce qui nous donne un score f1 de 0.9648664769170766


#### TF-IDF ngram_range=(1,3) && max_feature=80%


In [0]:
vectorizer_138 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3), max_features=octo_percent)

In [0]:
vectorized_corpus_138 = vectorizer_138.fit_transform(corpus)

In [0]:
svc_38 = LinearSVC()

In [0]:
accuracy = cross_val_score(svc_38, vectorized_corpus_138, y=target, scoring='accuracy', cv=3, n_jobs=-1)
recall = cross_val_score(svc_38, vectorized_corpus_138, y=target, scoring='recall', cv=3, n_jobs=-1)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
print(f"La precision du modèle est {np.mean(accuracy)} | la pertinence {np.mean(recall)} ce qui nous donne un score f1 de {2* np.mean(recall)*np.mean(accuracy)/(np.mean(recall)+np.mean(accuracy))}")

La precision du modèle est 0.9550599610411918 | la pertinence 0.968768074031232 ce qui nous donne un score f1 de 0.9618651793972717


#### TF-IDF(ngram_range=(1,3) && max_feature=3%) | TF-IDF(ngram_range=(1,3) && max_feature=40000 analyzer='char')


In [0]:
three_percent = math.ceil(vocab_size * 0.03)

In [0]:
vectorizer_133 = TfidfVectorizer(analyzer='word', tokenizer=prepareText, ngram_range=(1,3), max_features=20000)
vectorizer_c_133 = TfidfVectorizer(analyzer='char', ngram_range=(2,6), max_features=40000)

In [0]:
%%time
vectorized_corpus_133 = vectorizer_133.fit_transform(corpus)
vectorized_c_corpus_133 = vectorizer_c_133.fit_transform(corpus)

CPU times: user 52min 25s, sys: 48.8 s, total: 53min 13s
Wall time: 53min 14s


In [0]:
from scipy import sparse

In [0]:
wc_vect_corpus = sparse.hstack([vectorized_corpus_133, vectorized_c_corpus_133])

In [0]:
svc_33 = LinearSVC()

In [0]:
accuracy = cross_val_score(svc_33, wc_vect_corpus, y=target, scoring='accuracy', cv=3, n_jobs=-1)
recall = cross_val_score(svc_33, wc_vect_corpus, y=target, scoring='recall', cv=3, n_jobs=-1)

In [0]:
print(f"La precision du modèle est {np.mean(accuracy)} | la pertinence {np.mean(recall)} ce qui nous donne un score f1 de {2* np.mean(recall)*np.mean(accuracy)/(np.mean(recall)+np.mean(accuracy))}")

La precision du modèle est 0.9894513751256612 | la pertinence 0.9918064391748603 ce qui nous donne un score f1 de 0.9906275074518884


### Interprétation des résulats

* En ajoutant les ngram nous avons amélioré les performances de notre modèle. Nous somme passé de 95%  de f1-score (ngram_range=(1,1)) à 96% (ngram_range=(1,3)).

* Le couplage des deux TFIDF nous à permit de gagné 3% en précision. Ce qui est non négligeable. Le f1-score de notre modèle a atteint les 99% de f1-score. 

Il est donc claire qu'en influent sur ces deux paramètres que sont max_feature et ngram_range, nous améliorons les performances de notre modèle. Le max_feature, permet de réduire la complexité des données sur lequels notre modèle s'entraine en limitant le vocabulaire aux mots importants. Notre modèle se base donc uniquement sur ces mots pour faire ces prédictions. Le ngram_range, quant à lui ajoute de l'information, en mettant en évidence des groupes de tokens qui joueraient un rôle de thème.

Note : *Il est également possible d'influer sur les paramètres min_df, max_df. Ils permettent de supprimer les mots qui n'apparaissent pas plus de **min_df** fois ou qui apparaissent plus de **max_df** fois dans les textes.*

Bon maintenant nous allons valider notre modèle sur le dataset de test. Voir lesquelles généralise le mieux.

## Validation

Nous allons soumettre nos résultat sur kaggle afin de voir le score que l'on obtient avec notre meilleur modèle.

In [0]:
tfidf_word_svc = TfidfVectorizer(tokenizer=prepareText, analyzer='word', max_features=20000, ngram_range=(1,3))
tfidf_char_svc = TfidfVectorizer(analyzer='char', max_features=40000, stop_words='english', ngram_range=(2,6))
tfidf_svc = FeatureUnion([('word', tfidf_word_svc), ('char', tfidf_char_svc)])

In [0]:
svc_pip = make_pipeline(tfidf_svc, svc)

In [0]:
%%time
svc_pip.fit(corpus, target)

In [0]:
predictions_svc = svc_pip.predict(test_corpus)

### Submit

In [0]:
submit_svc = pd.DataFrame({'id': test_df.id, 'label' : predictions_svc})

In [0]:
submit_svc.to_csv('submission.csv', index=False)

In [0]:
!kaggle competitions submit -c fake-news -f submission.csv -m "SVC with default config and TFIDF"

![Texte alternatif…](https://i.ibb.co/qnp0y0q/Capture-d-cran-2020-05-16-08-05-59.png)

![Texte alternatif…](https://i.ibb.co/YRjWg5S/kaggle-result.png)

![Texte alternatif…](https://i.ibb.co/WHVyk53/Capture-d-cran-2020-05-16-08-08-40.png)

On obtient 99.2 comme private score et 99.1 en public score.

Remarque: L'usage du tfidf avec l'analyseur char peut rendre l'interprétation des prédictions moins lisible.

voir : https://colab.research.google.com/drive/14-sxbLTVi3MG-xFeywv-zpcmFc7QxkkN#scrollTo=3_ylaFCDwgLn